## Day 39 Proyect: CheapFlightTracker

### Proyecto Día 39 – Alerta de Vuelos Baratos ✈️📉

Este proyecto busca vuelos internacionales más baratos que los precios registrados en una hoja de cálculo de Google Sheets.

🚀 ¿Qué hace?

- Usa Amadeus API para consultar vuelos desde hoy hasta 6 meses.
- Compara precios actuales vs. precios mínimos definidos por el usuario.
- Envía una notificación por **WhatsApp** si se encuentra una buena oferta.

📦 Tecnologías y conceptos utilizados:

- API de Amadeus para búsqueda de vuelos
- API de Sheety para acceso a Google Sheets
- Envío de mensajes con Twilio WhatsApp
- Formateo de fechas, manejo de tokens y JSON complejos
- Programación orientada a objetos: `FlightSearch`, `FlightData`, `DataManager`, `NotificationManager`

In [1]:
import os
import requests
from datetime import datetime, timedelta
import time
from dotenv import load_dotenv
from requests.auth import HTTPBasicAuth
import re                                   # La libreria para expresiones regulares
from twilio.rest import Client

In [2]:
# Carga las variables del archivo .env
load_dotenv()  

True

#### Clase FlightSearch – Consulta de vuelos e IATA con Amadeus

In [ ]:
IATA_ENDPOINT = "https://test.api.amadeus.com/v1/reference-data/locations/cities"
FLIGHT_ENDPOINT = "https://test.api.amadeus.com/v2/shopping/flight-offers"
TOKEN_ENDPOINT = "https://test.api.amadeus.com/v1/security/oauth2/token"

# Datos de autenticación 
AMADEUS_API_KEY = os.getenv("AMADEUS_API_KEY")
AMADEUS_SECRET = os.getenv("AMADEUS_SECRET")

In [4]:
class FlightSearch:

    def __init__(self):
        """
        Inicializa una instancia de la clase FlightSearch.
        Variables de instancia:
        _api_key (str): La clave API para autenticación con Amadeus.
        _api_secret (str): El secreto API para autenticación con Amadeus.
        _token (str): El token de autenticación obtenido llamando al método _get_new_token.
        """
        self._api_key = AMADEUS_API_KEY
        self._api_secret = AMADEUS_SECRET

        # Obtener el token de acceso
        self._token = self._get_new_token()

    def _get_new_token(self):
        """
        Genera y devuelve el token de autenticación para acceder a la API de Amadeus.
        Esta función realiza una solicitud POST al endpoint de tokens de Amadeus con las credenciales
        necesarias (clave y secreto) para obtener un nuevo token de tipo client_credentials.
        Retorna:
            str: El nuevo token de acceso obtenido de la respuesta de la API.
        """
        # Encabezados para una solicitud autenticada
        header = {
            'Content-Type': 'application/x-www-form-urlencoded'
        }

        # Parámetros para la solicitud
        body = {
            'grant_type': 'client_credentials',
            'client_id': self._api_key,
            'client_secret': self._api_secret
        }

        # Realiza la solicitud POST al endpoint de Amadeus para obtener el token
        response = requests.post(url=TOKEN_ENDPOINT
                                ,headers=header
                                ,data=body)

        # Imprime el token de acceso
        #print(f"Access token: {response.json()['access_token']}")

        # Devuelve el token de acceso
        return response.json()['access_token']

    def get_destination_code(self, city_name):
        """
        Recupera el código IATA para una ciudad específica usando la API de Localización de Amadeus.
        Parámetros:
            city_name (str): Nombre de la ciudad para la cual se desea obtener el código IATA.
        Retorna:
            str: El código IATA de la primera ciudad coincidente si se encuentra; "N/A" si ocurre un IndexError,
            o "Not Found" si ocurre un KeyError.
        """
        # Encabezados para una solicitud autenticada
        headers = {"Authorization": f"Bearer {self._token}"}

        # Parámetros para la solicitud
        query = {
            "keyword": city_name,
            "max": "2",
            "include": "AIRPORTS",
        }

        # Realiza la solicitud GET al endpoint para obtener el código IATA
        response = requests.get(url=IATA_ENDPOINT
                               ,headers=headers
                               ,params=query)

        # Manejo de errores de respuesta
        try:
            code = response.json()["data"][0]["iataCode"]
            print(f'Código IATA: {response.json()["data"][0]["iataCode"]}')
            print(f'Código del aeropuerto: {response.json()["data"][0]["relationships"][0]["id"]}')
            print(f'Código del país: {response.json()["data"][0]["address"]["countryCode"]}')
        except IndexError:
            print(f"IndexError: No se encontró código de aeropuerto para {city_name}.")
            return "N/A"
        except KeyError:
            print(f"KeyError: No se encontró código de aeropuerto para {city_name}.")
            return "Not Found"

        return code

    def check_flights(self, origin_city_code, destination_city_code, from_time, to_time):
        """
        Busca opciones de vuelo entre dos ciudades en fechas de salida y regreso especificadas
        utilizando la API de Amadeus.
        Parámetros:
            origin_city_code (str): Código IATA de la ciudad de origen.
            destination_city_code (str): Código IATA de la ciudad de destino.
            from_time (datetime): Fecha de salida.
            to_time (datetime): Fecha de regreso.
        Retorna:
            dict o None: Un diccionario con los datos de las ofertas de vuelo si la consulta tiene éxito;
            None si ocurre un error.
        """
        # Encabezados para una solicitud autenticada
        headers = {"Authorization": f"Bearer {self._token}"}

        # Parámetros para la solicitud
        query = {
            "originLocationCode": origin_city_code,
            "destinationLocationCode": destination_city_code,
            "departureDate": from_time.strftime("%Y-%m-%d"),
            "returnDate": to_time.strftime("%Y-%m-%d"),
            "adults": 1,
            # "nonStop": "true",
            "currencyCode": "MXN",
            "max": "10",
        }

        # Realiza la solicitud GET al endpoint de Amadeus para obtener ofertas de vuelo
        response = requests.get(url=FLIGHT_ENDPOINT
                               ,headers=headers
                               ,params=query)

        # Imprime el código de estado si la respuesta no fue exitosa
        if response.status_code != 200:
            print(f"check_flights() código de respuesta: {response.status_code}")
            print("Cuerpo de la respuesta:", response.text)
            return None

        return response.json()


#### Clase FlightData – Representación de un vuelo

In [5]:
class FlightData:
    def __init__(self, price, origin_airport, destination_airport, out_date, return_date):
        """
        Constructor for initializing a new flight data instance with specific travel details.

        Parameters:
        - price: The cost of the flight.
        - origin_airport: The IATA code for the flight's origin airport.
        - destination_airport: The IATA code for the flight's destination airport.
        - out_date: The departure date for the flight.
        - return_date: The return date for the flight.
        """
        self.price = price
        self.origin_airport = origin_airport
        self.destination_airport = destination_airport
        self.out_date = out_date
        self.return_date = return_date

In [6]:
def find_cheapest_flight(data, max_stops=1, require_checked_bag=True):
    """
    Encuentra y muestra el vuelo más barato según los filtros dados: número máximo de escalas y
    si se requiere equipaje documentado.

    Args:
        data (dict): JSON con ofertas de vuelos desde la API de Amadeus.
        max_stops (int): Número máximo de escalas permitidas para considerar un vuelo.
        require_checked_bag (bool): Si True, solo se consideran vuelos que incluyen equipaje documentado.

    Returns:
        FlightData: Objeto con los detalles del vuelo más barato si se encuentra uno, o con 'N/A' si no.
    """
    if not data or not data.get('data'):
        print("No flight data")
        return FlightData("N/A", "N/A", "N/A", "N/A", "N/A")

    try:
        # Filtrar vuelos según los criterios
        filtered_offers = [ flight for flight in data["data"]
            if len(flight["itineraries"][0]["segments"]) - 1 <= max_stops and
               (not require_checked_bag or flight.get("pricingOptions", {}).get("includedCheckedBagsOnly", False))
        ]

        if not filtered_offers:
            print("No se encontraron vuelos con los filtros aplicados.")
            return FlightData("N/A", "N/A", "N/A", "N/A", "N/A")

        # Seleccionar el vuelo más barato
        cheapest_flight = min(filtered_offers
                            , key=lambda flight: float(flight["price"]["grandTotal"]))

        # Convertir duración a horas decimales
        def parse_duration_to_hours(duration_str):
            """Convierte duración tipo 'PT8H44M' a horas decimales"""
            hours = minutes = 0
            match_hours = re.search(r'(\d+)H', duration_str)
            match_minutes = re.search(r'(\d+)M', duration_str)
            if match_hours:
                hours = int(match_hours.group(1))
            if match_minutes:
                minutes = int(match_minutes.group(1))
            return round(hours + minutes / 60, 2)
        
        # Procesar itinerarios
        def process_itinerary(itinerary):
            segments = itinerary["segments"]
            first_segment = segments[0]
            last_segment = segments[-1]
            path = " → ".join([seg["departure"]["iataCode"] for seg in segments] + [last_segment["arrival"]["iataCode"]])
            stops = len(segments) - 1
            stops_text = f"{stops} escala" if stops == 1 else ("Vuelo Directo" if stops == 0 else f"{stops} escalas")
            duration = parse_duration_to_hours(itinerary["duration"])
            return first_segment["departure"]["iataCode"], last_segment["arrival"]["iataCode"], path, stops_text, duration

        # Itinerario de ida
        origin, destination, out_path, out_stops_text, out_duration = process_itinerary(cheapest_flight["itineraries"][0])
        out_date = cheapest_flight["itineraries"][0]["segments"][0]["departure"]["at"].split("T")[0]

        # Itinerario de regreso
        _, _, return_path, return_stops_text, return_duration = process_itinerary(cheapest_flight["itineraries"][1])
        return_date = cheapest_flight["itineraries"][1]["segments"][0]["departure"]["at"].split("T")[0]

        # Datos adicionales
        lowest_price = float(cheapest_flight["price"]["grandTotal"])
        airline = cheapest_flight["validatingAirlineCodes"][0]
        includes_checked_bag = cheapest_flight.get("pricingOptions", {}).get("includedCheckedBagsOnly", False)
        checked_bag_text = "✅ Sí incluye equipaje documentado" if includes_checked_bag else "❌ No incluye equipaje documentado"

        # Imprimir resultados
        print(f"✈️ Vuelo más barato a {destination}: ${lowest_price}")
        print(f"📍 Origen: {origin}")
        print(f"🗓️ Fecha de salida: {out_date} {out_path} ({out_stops_text})")
        print(f"⏱️ Duración total (salida): {out_duration}")
        print(f"🗓️ Fecha de regreso: {return_date} {return_path} ({return_stops_text})")
        print(f"⏱️ Duración total (regreso): {return_duration}")
        print(f"🛩️ Aerolínea: {airline}")
        print(checked_bag_text)

        return FlightData(lowest_price, origin, destination, out_date, return_date)

    except (KeyError, IndexError, ValueError) as e:
        print(f"Error processing flight data: {e}")
        return FlightData("N/A", "N/A", "N/A", "N/A", "N/A")

#### Clase DataManager – Lectura y escritura de Google Sheets vía Sheety


In [ ]:
# Datos de autenticación 
SHEET_ENDPOINT = os.getenv("SHEET_ENDPOINT")
SHEET_TOKEN = os.getenv("SHEET_TOKEN")

In [8]:
class DataManager:

    def __init__(self):
        """
        Inicializa una instancia de la clase DataManager.
        Esta clase se encarga de gestionar la comunicación con la API de Sheety para
        obtener y actualizar datos en una hoja de cálculo de Google Sheets.
        """
        self.destination_data = {}

    def get_destination_data(self):
        # Headers para una solicitud autenticada
        headers_sheety = {
            "Authorization": f"Bearer {SHEET_TOKEN}"
        }

        # Realiza una solicitud GET al endpoint de Sheety para obtener los datos de destinos
        response = requests.get(url=SHEET_ENDPOINT
                               ,headers=headers_sheety)

        # Guarda los datos de destinos en la variable de instancia
        self.destination_data = response.json()["prices"]  # "prices" es el nombre de la hoja en la hoja de cálculo

        # Devuelve los datos de destinos
        print(f"Destination data: {self.destination_data}")
        return self.destination_data

    def update_destination_codes(self):
        for city in self.destination_data:
            # Crea un diccionario con los datos actualizados (código IATA)
            new_data = {
                "price": {
                    "iataCode": city["iataCode"]
                }
            }

            # Realiza una solicitud PUT al endpoint de Sheety para actualizar el código IATA
            response = requests.put(url=f"{SHEET_ENDPOINT}/{city['id']}"
                                    ,json=new_data
                                    ,headers=headers_sheety)

            # Imprime la respuesta del servidor
            print(response.text)


#### Clase NotificationManager – Envío de mensajes vía Twilio WhatsApp


In [ ]:
# Twilio credentials 
TWILIO_SID = os.getenv("TWILIO_SID")
TWILIO_AUTH_TOKEN = os.getenv("TWILIO_AUTH_TOKEN")

# Sandbox de WhatsApp
WHATSAPP_FROM = "whatsapp:+1415523xxxx"   # Número de Twilio Sandbox
WHATSAPP_TO = "whatsapp:+521556696xxxx"   # Tu número verificado con prefijo

In [10]:
class NotificationManager:

    def __init__(self):
        """
        Inicializa una instancia del NotificationManager.

        Esta clase se encarga de enviar mensajes de WhatsApp utilizando la API de Twilio.
        Configura el cliente con las credenciales necesarias y define los números de
        remitente y destinatario.
        """
        self.client = Client(TWILIO_SID, TWILIO_AUTH_TOKEN)
        self.from_whatsapp = WHATSAPP_FROM
        self.to_whatsapp = WHATSAPP_TO

    def send_whatsapp(self, message):
        # Enviar un mensaje de WhatsApp utilizando la API de Twilio
        msg = self.client.messages.create(
            from_=self.from_whatsapp
            ,to=self.to_whatsapp
            ,body=message
            
        )
        
        print(f"✅ Mensaje enviado:\n{msg.body}")
        #print(f"Sid: {msg.sid}")


#### Paso 1: Obtener y actualizar datos del Google Sheet 🧾

Usamos Sheety para leer una hoja con precios mínimos de cada ciudad.  
Si no hay código IATA, se obtiene usando la API de Amadeus y se actualiza el documento.


In [11]:
# Inicializa las instancias necesarias para manejar datos y buscar vuelos
data_manager = DataManager()
flight_search = FlightSearch()

# Obtiene los datos de destinos desde la hoja de cálculo (Google Sheets)
sheet_data = data_manager.get_destination_data()

# # Recorre cada fila para verificar si falta el código IATA
# for row in sheet_data:
#     if row["iataCode"] == "":
#         # Si falta, consulta el código IATA usando la API de búsqueda de vuelos
#         row["iataCode"] = flight_search.get_destination_code(row["city"])
#         time.sleep(2)  # Pausa para evitar superar el límite de solicitudes de la API de Amadeus

# # Actualiza el atributo con los datos completos
# data_manager.destination_data = sheet_data

# # Envía los nuevos códigos IATA actualizados a la hoja de cálculo
# data_manager.update_destination_codes()


Destination data: [{'city': 'Paris', 'iataCode': 'CDG', 'lowestPrice': 15000, 'id': 2}, {'city': 'Medellin', 'iataCode': 'MDE', 'lowestPrice': 6000, 'id': 3}, {'city': 'Punta Cana', 'iataCode': 'PUJ', 'lowestPrice': 6000, 'id': 4}, {'city': 'Cusco', 'iataCode': 'LIM', 'lowestPrice': 6000, 'id': 5}, {'city': 'San Francisco', 'iataCode': 'SFO', 'lowestPrice': 15000, 'id': 6}, {'city': 'Tokyo', 'iataCode': 'NRT', 'lowestPrice': 15000, 'id': 7}]


#### Paso 2 – Buscar vuelos económicos con Amadeus 🛫

Usamos un rango de fechas desde mañana hasta dentro de 6 meses.  
Si el precio actual es menor al registrado, lo marcamos como oferta.


In [12]:
# Código IATA de la ciudad de origen (en este caso, Ciudad de México)
ORIGIN_CITY_IATA = "MEX"

# Fecha de inicio y de fin para la búsqueda de vuelos
tomorrow = datetime.now() + timedelta(days=1)
n_month_from_today = datetime.now() + timedelta(days=(3 * 30))

# Número máximo de escalas permitidas en el vuelo de ida
MAX_STOPS = 2                # 0 para vuelo directo, 1 para una escala, etc.
REQUIRE_CHECKED_BAG = False   # Si True, solo incluye vuelos que tengan equipaje documentado

# Lista para almacenar las mejores ofertas encontradas
deals = []

# Iterar sobre cada destino en los datos de la hoja
for destination in sheet_data:
    print(f"Buscando vuelos para {destination['city']}...")

    # Buscar vuelos desde la ciudad de origen al destino especificado, entre las fechas definidas
    flights = flight_search.check_flights(
                                origin_city_code=ORIGIN_CITY_IATA
                                ,destination_city_code=destination["iataCode"]
                                ,from_time=tomorrow
                                ,to_time=n_month_from_today
    )

    # Identificar el vuelo más barato dentro de los resultados obtenidos
    cheapest_flight = find_cheapest_flight(data=flights
                                        , max_stops=MAX_STOPS
                                        , require_checked_bag=REQUIRE_CHECKED_BAG)

    # Si hay un precio válido y es más bajo que el precio mínimo registrado, se agrega a las ofertas
    if cheapest_flight.price != "N/A" and cheapest_flight.price < destination["lowestPrice"]:
        deals.append(cheapest_flight)
        print(f"✈️ Nueva oferta encontrada: {cheapest_flight.price} MXN")


Buscando vuelos para Paris...
✈️ Vuelo más barato a CDG: $24638.0
📍 Origen: MEX
🗓️ Fecha de salida: 2025-05-12 MEX → BOG → CDG (1 escala)
⏱️ Duración total (salida): 17.83
🗓️ Fecha de regreso: 2025-08-09 CDG → BOG → MEX (1 escala)
⏱️ Duración total (regreso): 17.33
🛩️ Aerolínea: AV
✅ Sí incluye equipaje documentado
Buscando vuelos para Medellin...
✈️ Vuelo más barato a MDE: $9063.0
📍 Origen: MEX
🗓️ Fecha de salida: 2025-05-12 MEX → BOG → CLO → MDE (2 escalas)
⏱️ Duración total (salida): 8.73
🗓️ Fecha de regreso: 2025-08-09 MDE → MEX (Vuelo Directo)
⏱️ Duración total (regreso): 4.42
🛩️ Aerolínea: AV
✅ Sí incluye equipaje documentado
Buscando vuelos para Punta Cana...
✈️ Vuelo más barato a PUJ: $12212.0
📍 Origen: MEX
🗓️ Fecha de salida: 2025-05-12 MEX → BOG → PUJ (1 escala)
⏱️ Duración total (salida): 9.33
🗓️ Fecha de regreso: 2025-08-09 PUJ → MDE → MEX (1 escala)
⏱️ Duración total (regreso): 13.75
🛩️ Aerolínea: AV
✅ Sí incluye equipaje documentado
Buscando vuelos para Cusco...
✈️ Vuelo 

#### Paso 3 – Notificar al usuario vía WhatsApp si hay una oferta 🛩️📱

Si hay un vuelo con mejor precio que el registrado, enviamos una alerta usando Twilio.


In [13]:
# Crear una instancia del gestor de notificaciones, encargado de enviar mensajes por WhatsApp
notifier = NotificationManager()

# Iterar sobre cada vuelo encontrado que cumple con los criterios (mejores ofertas)
for flight in deals:
    # Construir el mensaje de alerta con los detalles del vuelo
    message = (
        f"Oferta detectada! ✈️\n"
        f"📍 {flight.origin_airport} ➡️ {flight.destination_airport}\n"
        f"💵 Precio: ${flight.price}\n"
        f"🗓️ Ida: {flight.out_date} | Regreso: {flight.return_date}"
    )

    # Enviar el mensaje por WhatsApp usando la clase NotificationManager
    notifier.send_whatsapp(message)


✅ Mensaje enviado:
Oferta detectada! ✈️
📍 MEX ➡️ SFO
💵 Precio: $10710.0
🗓️ Ida: 2025-05-12 | Regreso: 2025-08-09


#### 🧠 Conclusión

Este proyecto abarca automatización práctica con múltiples componentes reales:

- ✈️ Obtención de vuelos reales con API Amadeus
- 📊 Consulta y escritura en hojas de cálculo (Google Sheets vía Sheety)
- 🔐 Seguridad con `.env` y autenticación básica
- 📩 Alertas en tiempo real por WhatsApp
- 🧱 Estructura basada en clases y separación de responsabilidades

Es una base sólida para construir sistemas más avanzados como bots de viajes, sistemas de alertas o integraciones con aplicaciones móviles.
